In [40]:
"""
Extract TreeMap (c.a. 2016) statistics in active fire detections

"""

import os, sys
import ee
import geemap
import time
import pandas as pd

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()

ee.Initialize(project='jfsp-aspen')

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Success")

Success


In [4]:
# Load the USFS TreeMap
treemap = ee.ImageCollection("USFS/GTAC/TreeMap/v2016")
print(f"TreeMap bands available for analysis:\n\n{treemap.first().bandNames().getInfo()}")

TreeMap bands available for analysis:

['ALSTK', 'BALIVE', 'CANOPYPCT', 'CARBON_D', 'CARBON_DWN', 'CARBON_L', 'DRYBIO_D', 'DRYBIO_L', 'FLDSZCD', 'FLDTYPCD', 'FORTYPCD', 'GSSTK', 'QMD_RMRS', 'SDIPCT_RMRS', 'STANDHT', 'STDSZCD', 'TPA_DEAD', 'TPA_LIVE', 'Value', 'VOLBFNET_L', 'VOLCFNET_D', 'VOLCFNET_L']


In [ ]:
bands = ['FORTYPCD','BALIVE','CANOPYPCT']
treemap = treemap.select(bands)

In [15]:
class_codes = treemap.first().get('FORTYPCD_class_values').getInfo()
class_names = treemap.first().get('FORTYPCD_class_names').getInfo()
code_to_name = dict(zip(class_codes, class_names))
print(code_to_name)

{101: 'Jack pine', 102: 'Red pine', 103: 'Eastern white pine', 104: 'Eastern white pine / eastern hemlock', 105: 'Eastern hemlock', 121: 'Balsam fir', 122: 'White spruce', 123: 'Red spruce', 124: 'Red spruce / balsam fir', 125: 'Black spruce', 126: 'Tamarack', 127: 'Northern white-cedar', 141: 'Longleaf pine', 142: 'Slash pine', 161: 'Loblolly pine', 162: 'Shortleaf pine', 163: 'Virginia pine', 164: 'Sand pine', 165: 'Table Mountain pine', 166: 'Pond pine', 167: 'Pitch pine', 171: 'Eastern redcedar', 182: 'Rocky Mountain juniper', 184: 'Juniper woodland', 185: 'Pinyon / juniper woodland', 201: 'Douglas-fir', 202: 'Port-Orford-cedar', 221: 'Ponderosa pine', 222: 'Incense-cedar', 224: 'Sugar pine', 225: 'Jeffrey pine', 226: 'Coulter pine', 241: 'Western white pine', 261: 'White fir', 262: 'Red fir', 263: 'Noble fir', 264: 'Pacific silver fir', 265: 'Engelmann spruce', 266: 'Engelmann spruce / subalpine fir', 267: 'Grand fir', 268: 'Subalpine fir', 269: 'Blue spruce', 270: 'Mountain hemlo

In [20]:
# Save this file out.
out_fp = os.path.join(projdir,'data/tabular/mod/treemap_fortypcd_species_mapping.csv')
df = pd.DataFrame(list(code_to_name.items()), columns=['FORTYPCD', 'SpeciesName'])
print(f"Dictionary saved to {out_fp}")

Dictionary saved to /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/tabular/mod/treemap_fortypcd_species_mapping.csv


In [41]:
# Load the Active Fire Detections (AFD)
afds = ee.FeatureCollection('projects/jfsp-aspen/assets/AFD/combined-afd_aspen-fires_2018_to_2023')
afds = afds.filter(ee.Filter.eq('na_l3name', 'Southern Rockies'))
print(afds.size().getInfo())
print(afds.first().propertyNames().getInfo())

77339
['fid', 'FRP', 'NIFC_NAME', 'LONGITUDE', 'ACQ_YEAR', 'ACQ_TIME', 'SCAN', 'BRIGHTNESS', 'CONFIDENCE', 'TYPE', 'SATELLITE', 'na_l3name', 'DAYNIGHT', 'TRACK', 'INSTRUMENT', 'VID', 'NIFC_ID', 'START_YEAR', 'ACQ_DATETI', 'ACQ_MONTH', 'WF_CESSATI', 'VERSION', 'BRIGHT_T31', 'DISCOVERY_', 'afdID', 'LATITUDE', 'system:index', 'ACQ_DATE']


In [ ]:
# Perform reductions for the various bands

In [42]:
# Clean the property names before exporting reductions (just keep the ID)
afds_ = afds.select(['afdID'])

In [ ]:
# Histogram reducer for the tree code
imageC = treemap.select(['FORTYPCD']).first() # select just the forest type code

def species_histogram(ftr):
    hist = imageC.reduceRegion(
        reducer=ee.Reducer.frequencyHistogram(),
        geometry=ftr.geometry(),
        scale=30,
        maxPixels=1e13
    ).get('FORTYPCD')

    hist_dict = ee.Dictionary(hist)
    flat_ftr = hist_dict.keys().iterate(
        lambda key, f: ee.Feature(f).set(ee.String(key), hist_dict.get(key)), ftr
    )
    return ee.Feature(flat_ftr)


# Apply this to the AFDs
fortypcd = afds_.map(species_histogram)

# Print a sample
sample = fortypcd.limit(1).getInfo()
properties = sample['features'][0]['properties']
df = pd.DataFrame([properties])
print("Sample DataFrame:")
print(df.head())

# Export to Drive
fortypcd = fortypcd.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

task = ee.batch.Export.table.toDrive(
    collection=fortypcd,
    description='afd_aspen-fires_treemap-fortypcd',
    fileFormat='CSV',
    folder='TreeMap'
)
# Start the export task
task.start()
print("Export to Google Drive started!")
# Monitor the task until it's finished
monitor_export(task, timeout=120) # print every X seconds

In [ ]:
# Reductions for the continuous bands (BALIVE and CANOPYPCT)
imageC = treemap.select(['BALIVE','CANOPYPCT']).first()

# Mean, standard deviation, and median
reducer_ = ee.Reducer.mean().combine(
    reducer2=ee.Reducer.stdDev(), sharedInputs=True).combine(
    reducer2=ee.Reducer.median(), sharedInputs=True)

# Run the reduction
tmap_reduc = imageC.reduceRegions(
    collection=afds_,
    reducer=reducer_, 
    scale=30
)

# Export table to Drive.

# Remove geometry column first
tmap_reduc = tmap_reduc.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=tmap_reduc,
    description='afd_aspen-fires_treemap-balive_canopypct',
    fileFormat='CSV', 
    fileNamePrefix='afd_aspen-fires_treemap-balive_canopypct',
    folder='TreeMap'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 120) # print every 120 seconds

Export to Earth Engine Asset started!
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
Waiting for export to finish..
	Patience young padawan.
